In [1]:
import json
import pandas as pd

In [2]:
# Ratings given

emoji_map = {
    "1️⃣":1, "2️⃣":2, "3️⃣":3, "4️⃣":4, "5️⃣":5,
  "6️⃣":6, "7️⃣":7, "8️⃣":8, "9️⃣":9, "🔟":10
}

MIN_COUNT = 5
     
with open("food.json", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for msg in data["messages"]:
    for reaction in msg["reactions"]:
        if reaction["emoji"]["raw"] in emoji_map.keys():
            rating = reaction["score"]
            for user in reaction["users"]:
                rows.append({
                    "critic": user["username_at_time"],
                    "rating": rating
                })


critic_df = pd.DataFrame(rows)

critic_df = (
    critic_df.groupby("critic")
    .agg(
        avg_given=("rating", "mean"),
        count=("rating", "count"),
        std=("rating", "std")
    )
)

critic_df["z_score"] = (
    (critic_df["avg_given"] - critic_df["avg_given"].mean())
    / critic_df["avg_given"].std()
)

critic_df.query(f"count >= {MIN_COUNT}").sort_values("std", ascending=False)

,avg_given,count,std,z_score
critic,,,,
keisernavdombaas,4.267606,142,3.443393,-1.069064
boratsagdiyev5149,6.694444,72,3.106735,0.048211
stoffe1,6.015625,64,3.047206,-0.264306
konrad_certified_rizzler,5.770492,183,3.031302,-0.377161
lokieee.,7.000000,5,2.915476,0.188883
coalhawk,6.410256,117,2.853171,-0.082624
lordloke01,6.571429,7,2.636737,-0.008424
truckforerbastian,6.894737,38,2.586759,0.140422
etr91,5.333333,9,2.500000,-0.578422


In [3]:
# Ratings received

from collections import defaultdict

emoji_map = {
    "1️⃣":1, "2️⃣":2, "3️⃣":3, "4️⃣":4, "5️⃣":5,
  "6️⃣":6, "7️⃣":7, "8️⃣":8, "9️⃣":9, "🔟":10
}

MIN_COUNT = 15
              
with open("food.json", encoding="utf-8") as f:
    data = json.load(f)


users = defaultdict(lambda: defaultdict(list))

for msg in data["messages"]:
    chef = msg["author"]["username_at_time"]
    for reaction in msg["reactions"]:
        raw = reaction["emoji"]["raw"]
        if raw in emoji_map:
            rating = emoji_map[raw]
            for user in reaction["users"]:
                critic = user["username_at_time"]
                users[chef][critic].append(rating)

rows = []
for chef, critics in users.items():
    for critic, ratings in critics.items():
        for r in ratings:
            rows.append({
                "chef": chef,
                "critic": critic,
                "rating": r
            })


chef_df = pd.DataFrame(rows)

chef_df = (
    chef_df.groupby("chef")
    .agg(
        avg_received=("rating", "mean"),
        count=("rating", "count"),
        std=("rating", "std"),
        var=("rating", "var")
    )
)

chef_df.query(f"count >= {MIN_COUNT}").sort_values("std", ascending=False)

,avg_received,count,std,var
chef,,,,
konrad_certified_rizzler,5.684211,38,3.647326,13.302987
jesper7177,6.367347,147,3.162454,10.001118
truckforerbastian,5.930000,100,3.085499,9.520303
ivisumon,5.604878,205,2.917993,8.514682
kulmann123,6.195489,133,2.734346,7.476646
boratsagdiyev5149,6.757143,140,2.724447,7.422610
lordloke01,5.619048,21,2.479439,6.147619
stoffe1,6.578313,83,2.384505,5.685865
coalhawk,6.211268,71,2.260945,5.111871


In [4]:
# Dishes made per chef

MIN_COUNT = 5

dishes_per_chef = defaultdict(int)

with open("food.json", encoding="utf-8") as f:
    data = json.load(f)


for msg in data["messages"]:
    chef = msg["author"]["username_at_time"]
    dishes_per_chef[chef] += 1

flattened = []
for el in dishes_per_chef:
    flattened.append({"chef": el, "dishes": dishes_per_chef[el]})

dish_df = (
    pd.DataFrame(flattened)
    .query(f"dishes >= {MIN_COUNT}")
    .sort_values("dishes", ascending=False)
    .reset_index(drop=True)
    .set_index("chef")
)

dish_df 

,dishes
chef,
ivisumon,64
jesper7177,50
boratsagdiyev5149,38
kulmann123,35
stoffe1,32
truckforerbastian,24
coalhawk,23
konrad_certified_rizzler,17
oddbjorn6810,5


In [5]:
import statistics

# Individual dishes

dishes = []

MIN_COUNT = 5

for msg in data["messages"]:
    
    ratings = [
        emoji_map[reaction["emoji"]["raw"]]
        for reaction in msg["reactions"]
        if reaction["emoji"]["raw"] in emoji_map
        for _ in reaction["users"]
    ]

    count = len(ratings)

    if count >= 2:
        avg_rating = statistics.mean(ratings)
        sd = statistics.stdev(ratings)
    elif count == 1:
        avg_rating = ratings[0]
        sd = 0.0
    else:
        avg_rating = None
        sd = None

    dishes.append({
        "dish": msg["content"] or "<no description>",
        "chef": msg["author"]["username_at_time"],
        "images": [img["url"] for img in msg["images"]],
        "ratings": ratings,
        "avg_rating": avg_rating,
        "sd": sd,
        "count": count
    })

df = (
    pd.DataFrame(dishes)
    .query(f"count >= {MIN_COUNT}")
    .sort_values("avg_rating", ascending=False)
    # .sort_values("avg_rating")
    # .sort_values("sd", ascending=False)
    # .sort_values("count", ascending=False)
)

df.head(10)

# for el in df.head(5)["images"]:
#     print(el)

,dish,chef,images,ratings,avg_rating,sd,count
80,<no description>,truckforerbastian,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 10, 10, 10]",10.000000,0.000000,5
240,<no description>,kulmann123,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 10, 10, 10]",10.000000,0.000000,5
102,Food pic,etr91,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 10, 10, 10]",10.000000,0.000000,5
255,<no description>,mrmelk,[https://cdn.discordapp.com/attachments/932063...,"[9, 9, 10, 10, 10]",9.600000,0.547723,5
85,<no description>,truckforerbastian,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 10, 10, 9, 9, 9]",9.571429,0.534522,7
34,Ramen,jesper7177,[https://cdn.discordapp.com/attachments/932063...,"[10, 9, 9, 9, 8]",9.000000,0.707107,5
154,<no description>,coalhawk,[https://cdn.discordapp.com/attachments/932063...,"[8, 8, 8, 10, 10]",8.800000,1.095445,5
68,<no description>,boratsagdiyev5149,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 8, 8, 8, 8]",8.666667,1.032796,6
10,<no description>,truckforerbastian,[https://cdn.discordapp.com/attachments/932063...,"[10, 10, 10, 10, 4, 8]",8.666667,2.422120,6
4,Saken er biff,boratsagdiyev5149,[https://cdn.discordapp.com/attachments/932063...,"[8, 8, 10, 9, 9, 7]",8.500000,1.048809,6


In [6]:
# rating of others compared to mean

emoji_map = {
    "1️⃣":1, "2️⃣":2, "3️⃣":3, "4️⃣":4, "5️⃣":5,
  "6️⃣":6, "7️⃣":7, "8️⃣":8, "9️⃣":9, "🔟":10
}

MIN_COUNT = 10
              
with open("food.json", encoding="utf-8") as f:
    data = json.load(f)

rows = []

for msg in data["messages"]:
    for reaction in msg["reactions"]:
        raw = reaction["emoji"]["raw"]
        if raw in emoji_map:
            rating = emoji_map[raw]
            for user in reaction["users"]:
                rows.append({
                    "cook": msg["author"]["username_at_time"],
                    "critic": user["username_at_time"],
                    "rating": rating
                })

df = pd.DataFrame(rows)

# baselines
global_avg = df["rating"].mean()
critic_avg = df.groupby("critic")["rating"].mean()
cook_avg = df.groupby("cook")["rating"].mean()

# pair stats
pair_avg = (
    df
    .groupby(["critic","cook"])
    .agg(
        mean_rating=("rating","mean"),
        count=("rating","count")
    )
    .reset_index()
)

# attach baselines to pairs
pair_avg["critic_avg"] = pair_avg["critic"].map(critic_avg)
pair_avg["cook_avg"] = pair_avg["cook"].map(cook_avg)

# bias
pair_avg["bias"] = (
    pair_avg["mean_rating"]
    - pair_avg["critic_avg"]
    - pair_avg["cook_avg"]
    + global_avg
)

pair_avg.query(f"count >= {MIN_COUNT}").sort_values("bias", ascending=False).head(10)


,critic,cook,mean_rating,count,critic_avg,cook_avg,bias
102,keisernavdombaas,kulmann123,8.615385,13,4.267606,6.195489,4.310083
150,stoffe1,jesper7177,8.285714,14,6.015625,6.367347,2.060535
20,coalhawk,boratsagdiyev5149,8.357143,14,6.410256,6.757143,1.347536
94,keisernavdombaas,boratsagdiyev5149,6.181818,22,4.267606,6.757143,1.314863
76,jesper7177,ivisumon,6.722222,54,6.336493,5.604878,0.938644
73,jesper7177,coalhawk,7.187500,16,6.336493,6.211268,0.797532
53,ivisumon,jesper7177,8.235294,34,7.365385,6.367347,0.660355
118,konrad_certified_rizzler,ivisumon,5.729730,37,5.770492,5.604878,0.512153
11,boratsagdiyev5149,kulmann123,7.285714,14,6.826087,6.195489,0.421931
89,jesper7177,truckforerbastian,6.421053,19,6.336493,5.930000,0.312353
